In [5]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
from models.DGC import DGC
from time import perf_counter

In [6]:
lr = 0.2
bias = False
epochs = 50
weight_decay = 5e-4
loss_fn = torch.nn.CrossEntropyLoss()

In [7]:
data = PubmedGraphDataset()
g = data[0]
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [8]:
model = DGC(in_feats, n_classes, K=2, T=2, bias=bias)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [9]:
def test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [10]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    training_time = perf_counter()-t
    print(f'Training time: {training_time:.4f}s')

In [11]:
train(model, g, features, labels, train_mask, val_mask, test_mask)
test(model, g, features, labels, test_mask)

Epoch 01, Loss: 1.0991, Train: 0.9167, Val: 0.7040, Test: 0.7010
Epoch 02, Loss: 1.0089, Train: 0.9333, Val: 0.7200, Test: 0.7310
Epoch 03, Loss: 0.9311, Train: 0.9500, Val: 0.7400, Test: 0.7430
Epoch 04, Loss: 0.8672, Train: 0.9500, Val: 0.7420, Test: 0.7530
Epoch 05, Loss: 0.8168, Train: 0.9667, Val: 0.7480, Test: 0.7490
Epoch 06, Loss: 0.7786, Train: 0.9667, Val: 0.7480, Test: 0.7420
Epoch 07, Loss: 0.7505, Train: 0.9667, Val: 0.7420, Test: 0.7450
Epoch 08, Loss: 0.7306, Train: 0.9667, Val: 0.7480, Test: 0.7460
Epoch 09, Loss: 0.7174, Train: 0.9667, Val: 0.7520, Test: 0.7460
Epoch 10, Loss: 0.7094, Train: 0.9667, Val: 0.7600, Test: 0.7480
Epoch 11, Loss: 0.7052, Train: 0.9667, Val: 0.7640, Test: 0.7490
Epoch 12, Loss: 0.7038, Train: 0.9500, Val: 0.7720, Test: 0.7530
Epoch 13, Loss: 0.7042, Train: 0.9500, Val: 0.7780, Test: 0.7580
Epoch 14, Loss: 0.7059, Train: 0.9500, Val: 0.7800, Test: 0.7600
Epoch 15, Loss: 0.7081, Train: 0.9500, Val: 0.7780, Test: 0.7620
Epoch 16, Loss: 0.7105, T

0.771